# Partitioning the input files

ROMS requires partitioned (or tiled) input files so that the simulation can be parallelized over multiple nodes. `ROMS-Tools` can create these partitioned files for you.

## Writing some example files
First we make some (non-partitioned) example files, which we will then partition in the next section. 

### Example grid file

In [1]:
from roms_tools import Grid

In [2]:
grid = Grid(
    nx=1000, ny=1000, size_x=1800, size_y=2400, center_lon=-21, center_lat=61, rot=20
)

In [3]:
grid.ds

<xarray.Dataset> Size: 107MB
Dimensions:       (eta_rho: 1002, xi_rho: 1002, xi_u: 1001, eta_v: 1001,
                   eta_coarse: 502, xi_coarse: 502, s_rho: 100, s_w: 101)
Coordinates:
    lat_rho       (eta_rho, xi_rho) float64 8MB 47.96 47.96 ... 73.42 73.42
    lon_rho       (eta_rho, xi_rho) float64 8MB 333.1 333.1 ... 352.9 353.0
    lat_u         (eta_rho, xi_u) float64 8MB 47.96 47.97 47.97 ... 73.42 73.42
    lon_u         (eta_rho, xi_u) float64 8MB 333.1 333.1 333.1 ... 352.9 353.0
    lat_v         (eta_v, xi_rho) float64 8MB 47.97 47.97 47.98 ... 73.41 73.41
    lon_v         (eta_v, xi_rho) float64 8MB 333.1 333.1 333.1 ... 352.9 353.0
    lat_coarse    (eta_coarse, xi_coarse) float64 2MB 47.94 47.96 ... 73.44
    lon_coarse    (eta_coarse, xi_coarse) float64 2MB 333.1 333.1 ... 353.0
Dimensions without coordinates: eta_rho, xi_rho, xi_u, eta_v, eta_coarse,
                                xi_coarse, s_rho, s_w
Data variables: (12/15)
    angle         (eta_rho, xi_rho) float64 8MB 0.4199 0.4199 ... 0.11 0.11
    f             (eta_rho, xi_rho) float64 8MB 0.000108 0.000108 ... 0.0001394
    pm            (eta_rho, xi_rho) float64 8MB 0.0004209 ... 0.0004209
    pn            (eta_rho, xi_rho) float64 8MB 0.0005593 ... 0.0005593
    spherical     |S1 1B b'T'
    mask_rho      (eta_rho, xi_rho) int32 4MB 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1
    ...            ...
    mask_coarse   (eta_coarse, xi_coarse) int32 1MB 1 1 1 1 1 1 ... 1 1 1 1 1 1
    h             (eta_rho, xi_rho) float64 8MB 2.965e+03 ... 3.108e+03
    sigma_r       (s_rho) float32 400B -0.995 -0.985 -0.975 ... -0.015 -0.005
    Cs_r          (s_rho) float32 400B -0.992 -0.9753 ... -8.89e-05 -9.874e-06
    sigma_w       (s_w) float32 404B -1.0 -0.99 -0.98 -0.97 ... -0.02 -0.01 0.0
    Cs_w          (s_w) float32 404B -1.0 -0.9837 -0.9667 ... -3.95e-05 0.0
Attributes:
    title:               ROMS grid created by ROMS-Tools
    roms_tools_version:  0.1.dev177
    size_x:              1800
    size_y:              2400
    center_lon:          -21
    center_lat:          61
    rot:                 20
    topography_source:   ETOPO5
    hmin:                5.0
    theta_s:             5.0
    theta_b:             2.0
    hc:                  300.0

In [4]:
grid.save("/pscratch/sd/n/nloose/grids/my_roms_grid.nc")

INFO - Writing the following NetCDF files:
/pscratch/sd/n/nloose/grids/my_roms_grid.nc


[PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.nc')]

### Example surface forcing file

In [5]:
from roms_tools import SurfaceForcing
from datetime import datetime

In [6]:
start_time = datetime(2012, 1, 1)
end_time = datetime(2012, 1, 31)

In [7]:
era5_path = "/global/cfs/projectdirs/m4746/Datasets/ERA5/NA/2012/ERA5_2012-01.nc"

In [8]:
surface_forcing = SurfaceForcing(
    grid=grid,
    start_time=start_time,
    end_time=end_time,
    source={"name": "ERA5", "path": era5_path},
    use_dask=True,
)

INFO - Data will be interpolated onto grid coarsened by factor 2.


In [9]:
surface_forcing.ds

<xarray.Dataset> Size: 5GB
Dimensions:   (time: 721, eta_rho: 502, xi_rho: 502)
Coordinates:
    abs_time  (time) datetime64[ns] 6kB 2012-01-01 ... 2012-01-31
  * time      (time) float64 6kB 4.383e+03 4.383e+03 ... 4.413e+03 4.413e+03
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    swrad     (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    lwrad     (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    Tair      (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    qair      (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    rain      (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    uwnd      (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
    vwnd      (time, eta_rho, xi_rho) float32 727MB dask.array<chunksize=(1, 502, 502), meta=np.ndarray>
Attributes:
    title:                 ROMS surface forcing file created by ROMS-Tools
    roms_tools_version:    0.1.dev177
    start_time:            2012-01-01 00:00:00
    end_time:              2012-01-31 00:00:00
    source:                ERA5
    correct_radiation:     True
    use_coarse_grid:       True
    model_reference_date:  2000-01-01 00:00:00
    type:                  physics

In [10]:
filepath_forcing = "/pscratch/sd/n/nloose/my_surface_forcing.nc"

In [11]:
surface_forcing.save(filepath_forcing)

INFO - Writing the following NetCDF files:
/pscratch/sd/n/nloose/my_surface_forcing_201201.nc


[########################################] | 100% Completed | 113.62 s


[PosixPath('/pscratch/sd/n/nloose/my_surface_forcing_201201.nc')]

## Partitioning existing files
We will now partition the two files written in the previous section. However, note that the `partition_netcdf` tool in `ROMS-Tools` operates independently of whether the files were originally created using `ROMS-Tools`.

In [12]:
from roms_tools import partition_netcdf

We need to tell the `partition_netcdf` function what domain decomposition to use via the following two parameters:

* `np_eta` : The number of partitions along the `eta` direction.
* `np_xi` : The number of partitions along the `xi` direction.

In [13]:
%time partition_netcdf("/pscratch/sd/n/nloose/grids/my_roms_grid.nc", np_eta=5, np_xi=5)

CPU times: user 467 ms, sys: 300 ms, total: 767 ms
Wall time: 1.65 s


[PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.0.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.1.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.2.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.3.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.4.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.5.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.6.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.7.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.8.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.9.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.13.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.14.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_roms_grid.15.nc'),
 PosixPath('/pscra

In [14]:
%time partition_netcdf('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.nc', np_eta=5, np_xi=5)

CPU times: user 2.53 s, sys: 19.1 s, total: 21.6 s
Wall time: 22.4 s


[PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.0.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.1.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.2.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.3.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.4.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.5.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.6.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.7.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.8.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.9.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.10.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.11.nc'),
 PosixPath('/pscratch/sd/n/nloose/grids/my_surface_forcing_201201.12.nc'),
 PosixPath('/pscratch/sd/n/nloose/g